In [1]:
## Set up

import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras.models import Model
%matplotlib inline

train_path = 'jaguar-and-notjaguar/train'
valid_path = 'jaguar-and-notjaguar/valid'
test_path = 'jaguar-and-notjaguar/test'

# check batch sizes are correct
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['jaguar', 'notjaguar'], batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['jaguar', 'notjaguar'], batch_size=5)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['jaguar', 'notjaguar'], batch_size=10)

# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows +1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

imgs, labels = next(train_batches)

plots(imgs, titles=labels)

# Build Fine-tuned VGG16 model

vgg16_model = keras.applications.vgg16.VGG16()

vgg16_model.summary()

#Get the last but one layer/tensor from the old model
last_layer = vgg16_model.layers[-2].output

#Define the new layer/tensor for the new model
new_model = Dense(2, activation='softmax', name='Binary_predictions')(last_layer)

#Create the new model, with the old models input and the new_model tensor as the output

new_model = Model(vgg16_model.input, new_model, name='Finetuned_VGG16')

#Set all layers,except the last one to not trainable
for layer in new_model.layers[:-1]: layer.trainable=False

new_model.summary()

# Train the fine-tuned VGG16 model

#now train with the new outputs (jaguar and notjaguar)

#Compile the new model
new_model.compile(Adam(lr=.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])

new_model.fit_generator(train_batches, steps_per_epoch=60, 
                        validation_data=valid_batches, validation_steps=60, epochs=5, verbose=2)

score = new_model.evaluate_generator(test_batches, steps=1)

test_loss = score[0]
test_acc = score[1]
print("%s: %.2f%%" % (new_model.metrics_names[1], score[1]*100))

Using TensorFlow backend.
